In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
with open('/content/drive/MyDrive/QA/train-v2.0.json') as f:
    data = json.load(f)

In [3]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:

# data
# data['data']
# type(data['data'])
# data['data'][0]
# data['data'][1]
# len(data['data'])
# type(data['data'][0])
# data['data'][0]['title']
# data['data'][0]['paragraphs']
# len(data['data'][0]['paragraphs'])
# data['data'][0]['paragraphs'][0]
# data['data'][0]['paragraphs'][1]
# data['data'][0]['paragraphs'][0]['qas']
# data['data'][0]['paragraphs'][0]['qas'][0]
# data['data'][0]['paragraphs'][0]['qas'][0]['question']
# data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]
# data['data'][0]['paragraphs'][0]['context']
# data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['text']
# len(data['data'])

In [5]:
questions = []
contexts = []
answers = []
for i in range(len(data['data'])):
    data_i = data['data'][i]
    for j in range(len(data_i['paragraphs'])):
        paragraphs_ij = data_i['paragraphs'][j]
        context = paragraphs_ij['context']
        qas = paragraphs_ij['qas']
        for l in range(len(qas)):
            question = qas[l]['question']
            if len(qas[l]['answers']) >= 1:
                answer = qas[l]['answers'][0]['text']
            else:
                answer = qas[l]['plausible_answers'][0]['text']
                print(question)
                print(answer)
            questions.append(question)
            contexts.append(context)
            answers.append(answer)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
 What doesn't Bellah use the term "civil religion' to describe?
specific relation between politics and religion in the United States
 Whose speech does Bellah's 1977 article analyze?
John F. Kennedy
 What word does Bellah ask how a President is justified to not use?
God
What has the separation of church and state failed to approve the political realm of?
religious dimension
Who hasn't argued the United States is a model for the world in separation of church and state is a good thing?
Robert S. Wood
What does having no state-run or state-established allow a variety of non-religions to do?
flourish
 Where did the Center for New Religions leave?
Toronto
 What does Wood believe the U.K. operates on?
a sort of civic religion
 What does Wood call the approach of allowing individuals the freedom to decide what they don't want to believe?
genius of religious sentiment in the United States
What Portugese District is the capital of Brazil?

In [6]:
print(len(answers))

130319


In [7]:
def find_sub_list(sl, l):
    sll = len(sl)
    for ind in (i for i, e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll] == sl:
            return ind, ind+sll-1

In [8]:
# !pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 24.1 MB/s 
     |████████████████████████████████| 120 kB 22.0 MB/s 
     |████████████████████████████████| 6.6 MB 57.2 MB/s 


In [9]:
from transformers import BertTokenizer, BertModel
import torch
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased").to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
# X_ids = []
# Y_se = []
# for i in range(len(questions)):
# # for i in range(60):
#     inputs_qc = tokenizer(questions[i], contexts[i], return_tensors='pt')
#     inputs_a = tokenizer(answers[i], return_tensors='pt')
#     if inputs_qc['input_ids'].shape[1] > 512:
#         continue
#     try:
#         start, end = find_sub_list(inputs_a['input_ids'].tolist()[0][1:-1], inputs_qc['input_ids'].tolist()[0])
#         X_ids.append(inputs_qc)
#         length = inputs_qc['input_ids'].shape[1]
#         Y_se.append(torch.tensor([start, end]))
#     except:
#         print('error')

In [11]:
# import pickle
# with open('/content/drive/MyDrive/X_ids.pickle', 'wb') as handle:
#     pickle.dump(X_ids, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('/content/drive/MyDrive/Y_se.pickle', 'wb') as handle:
#     pickle.dump(Y_se, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
import pickle
with open('/content/drive/MyDrive/X_ids.pickle', 'rb') as handle:
    X_ids = pickle.load(handle)
with open('/content/drive/MyDrive/Y_se.pickle', 'rb') as handle:
    Y_se = pickle.load(handle)

In [13]:
print(len(X_ids), len(Y_se))

129935 129935


In [14]:
print(len(X_ids))
print(len(Y_se))

129935
129935


In [15]:
import torch
import torch.nn as nn

In [16]:
class MyLayer(nn.Module):
    def __init__(self):
        super(MyLayer, self).__init__()
        self.Ws = nn.parameter.Parameter(torch.randn(768), requires_grad=True)
        self.We = nn.parameter.Parameter(torch.randn(768), requires_grad=True)

    def forward(self, inputs):
        outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        matrix = last_hidden_states[0]
        length = matrix.shape[0]
        matrix_start = torch.zeros(length)
        matrix_end = torch.zeros(length)
        for i in range(length):
            matrix_start[i] = torch.matmul(self.Ws, matrix[i])
            matrix_end[i] = torch.matmul(self.We, matrix[i])
        matrix_start_sm = nn.Softmax()(matrix_start)
        matrix_end_sm = nn.Softmax()(matrix_end)
        return torch.stack((matrix_start_sm, matrix_end_sm), 0)

In [17]:
import torch.optim as optim

# Instantiate the model
mymodel = MyLayer().to(device)

# Define the optimizer
optimizer = optim.SGD(mymodel.parameters(), lr=0.001, momentum=0.9)

# Define loss using a predefined loss function
criterion = nn.CrossEntropyLoss().to(device)

In [18]:
n_epoch = 10
best_loss = 1e9
checkpoint_interval = 2000
for epoch in range(n_epoch):
    running_loss = 0.0
    for i in range(10000):
        # get the inputs and the labels
        x = X_ids[i].to(device)
        y = Y_se[i].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        y_pred = mymodel(x)
        loss = criterion(y_pred.to(device), y)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        if i % checkpoint_interval == checkpoint_interval-1:
            print(f'[{epoch+1}, {i+1}] loss: {running_loss/checkpoint_interval}')
            loss = running_loss/checkpoint_interval
            if loss < best_loss:
                best_loss = loss
                torch.save({
                    "epoch": epoch,
                    "loss": best_loss,
                    "mymodel": mymodel
                }, '/content/drive/MyDrive/checkpoint.pt')
                print(best_loss)
            running_loss = 0.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[1, 2000] loss: 5.127275374412537
5.127275374412537
[1, 4000] loss: 4.9486661952733995
4.9486661952733995
[1, 6000] loss: 5.0064181543588635
[1, 8000] loss: 4.853888941168785
4.853888941168785
[1, 10000] loss: 4.748005045413971
4.748005045413971
[2, 2000] loss: 5.125662924766541
[2, 4000] loss: 4.946796455979348
[2, 6000] loss: 5.005273498415947
[2, 8000] loss: 4.8516701483726505
[2, 10000] loss: 4.746264396190643
4.746264396190643
[3, 2000] loss: 5.124236333489418
[3, 4000] loss: 4.945081619501114
[3, 6000] loss: 5.004070427417755
[3, 8000] loss: 4.849717431902885
[3, 10000] loss: 4.744293383598327
4.744293383598327
[4, 2000] loss: 5.1230779832601545
[4, 4000] loss: 4.943275419592857
[4, 6000] loss: 5.0027999794483184
[4, 8000] loss: 4.847816164135933
[4, 10000] loss: 4.742263140439987
4.742263140439987
[5, 2000] loss: 5.122006952881813
[5, 4000] loss: 4.941513337373734
[5, 6000] loss: 5.00142184817791
[5, 8000] loss: 4.845796997785568
[5, 10000] loss: 4.740300181269646
4.740300181269

In [ ]:
# checkpoint = torch.load('checkpoint.pt')
# model = checkpoint['mymodel']
# output = model(input)
# print(output) 

In [ ]:
# import torch

# best_loss = 1e6
# if loss < best_loss:
#     best_loss = loss
#     torch.save({
#         "n_epoch": n_epoch,
#         "loss": best_loss,
#         "mymodel": mymodel
#     },
#     "checkpoint.pt"
#     )
# checkpoint = torch.load("checkpoint.pt")
# print(checkpoint["model"])
# input = torch.randn(16, 3, 224, 224)
# output = model(input)
# print(output.shape)